In [74]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Importo librería de Carla

from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None) # Muestra todas las columnas de los DataFrame
pd.set_option('display.width', 1000) # Ajusta el ancho de la consola para evitar truncamientos


In [70]:
def generar_driver ():

    # Usaremos esta función para creara un único driver que reutilizaremos durante la búsqueda de datos.
    # Devuelve el driver que usaremos para hacer el web scrapping de las webs de las películas. 
    driver = webdriver.Chrome()

    sleep(2)
    driver.maximize_window()
    print("Maximizando la web.")
    sleep(2)
    driver.get("https://www.imdb.com/")
    print("Accediendo a la web.")
    sleep(2)
    try:
        driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
        print("Cookies aceptadas.")
        sleep(2)
    except:
        sleep(2)
    return driver

# ¿Dónde cerraremos este driver? Manualmente, después de haber llamado a la función extraer_actores (). 

In [114]:

def web_scrapping (driver, id_pelicula):

    # Esta función recibe un driver y un identificador de película, lo que le permite hacer web scrapping.
    # Devuelve una lista con los datos de los 10 intérpretes de la película.

    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"     # Construimos url con el identificador pasado por parámetro.
    driver.get(url_pelicula)
    sleep(2)
    nombre_pelicula = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div.sc-1f50b7c-0.iPPbjm > h1 > span").text
    print(f"Entrando en la web de la película {nombre_pelicula}: {url_pelicula}")
    sleep(2)

    # Inicializo el diccionario donde guardaré las URL y el nombre de cada intérprete.

    actores_url = {} 
    lista_actores = []

    # De la página de la película extraigo los nombres y urls de los 10 actores principales.

    try:
        elemento_casting = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid")
        print("Explorando el casting")
        sleep(2)

    except:
        print("No hemos podido acceder al casting de la película.")
        return
    

    for i in range (1, 11):
        try:
            actor = elemento_casting.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.kSFMrr > a")
            nombre_actor = actor.text
            # nombre.append(nombre_actor)
            actores_url[nombre_actor] = actor.get_attribute("href")
        except:
            print("No se ha encontrado o no se ha podido extraer la url de algún actor")
    print("Extraigo el nombre y url de los actores")
    sleep(2)

    # Ahora que tengo sus direcciones url, exploro cada web en busca de los datos restantes del actor.

    for actor, url in actores_url.items():
        print(f"Entrando en la url de {actor}")
        driver.get(url)
        sleep(2)
        print("\tExtraemos año de nacimiento")

        try: 
            fecha_completa = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-11.hFWIYv > section > aside > div > span:nth-child(2)").text
            lista_year = fecha_completa.split()
            year = lista_year[-1]
        except: 
            print("No se ha podido extraer el año del intérprete.")
            year = 0    # En caso de no haber podido extraer, devolverá el año 0.

        print(f"\tExtraemos otros roles en la industria")
    
        sleep(2)
        lista_roles = []
        roles = ""
        try:
            elemento_roles= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div > ul").find_elements("css selector", "li")
            for rol in elemento_roles:
                lista_roles.append(rol.text)
            roles = ". ".join(lista_roles)
        except:
            print("Ha ocurrido un error con los roles del intérprete.")
            print(f"Roles de interprete es un tipo {type(roles)}. Contiene ahora mismo:{roles} ")

                
        sleep(2)

        print("\tRecopilando otras películas por las que se le conoce")

        conocido_por = [] 

        try:
            selector_conocido_por = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-1c5b3162-0.dsjiSY > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid")
            otras_peliculas = selector_conocido_por.find_elements("css selector", "a") # Busca todos los enlaces que haya en la caja de conocido por.
            for pelicula in otras_peliculas:
                titulo = pelicula.text
                if len(titulo)>1:                   # Esta condición es para evitar que capture títulos vacíos, que es algo común en imdb.
                    conocido_por.append(titulo)   
        except:   
            print("Ha ocurrido un error durante la extracción de sus otras películas")

        otras_peliculas = ". ".join(conocido_por)
        

        print("\tRecopilando premios")
        # Al estar en la versión MPV, de momento nos vamos a quedar solo con el resumen de los premios. 
        resumen_premios = ""

        try: 
            resumen_premios = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link").text
            sleep(2)
            if resumen_premios == "Premios":
                resumen_premios = "Otros premios"

        except:
            print(f"{actor} no tiene resumen de premios")
            resumen_premios = "No tiene premios"
        

        # Creamos una tupla con los datos del actor recién explorado y la añadimos a la lista.
        
        tupla_actor = (actor, year, roles, otras_peliculas, resumen_premios)
        lista_actores.append(tupla_actor)       
    
    print("Finalizamos exploración de los actores principales")
    return lista_actores



In [94]:

def extraer_datos_actores (lista_ids):

    # Recibe una lista de identificadores de películas.
    # Devuelve un dataframe con los datos de los actores de esas pelicula.

    mi_driver = generar_driver()
    datos_actores_total = []

    for id in lista_ids:
        datos_actores = web_scrapping(mi_driver, id)     # Llamamos a nuestra función para sacar datos de esa película
        datos_actores_total.extend(datos_actores)        # Añadimos el resultado a una lista que contendrá todos los datos de la lista de identificadores.

    columnas = ["Nombre Intérprete", "Año de nacimiento", "Roles en la industria", "Conocido por", "Premios"]
    df_actores = pd.DataFrame(datos_actores_total, columns=columnas)
    mi_driver.close() 
    print("Cerrando navegador")
    print(df_actores)
    return df_actores


Hasta aquí, el código. A partir de este momento, hacemos pruebas y extracciones.

In [115]:
lista_identificadores1 = ['tt0122161']
data_frame_actores1 = extraer_datos_actores (lista_identificadores1)


Maximizando la web.
Accediendo a la web.
Cookies aceptadas.
Entrando en la web de la película Falsa inocencia: https://www.imdb.com/title/tt0122161/
Explorando el casting
Extraigo el nombre y url de los actores
Entrando en la url de Dennis Hopper
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Marley Shelton
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Devon Gummersall
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Talia Shire
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Michael Cudlitz
	Extraemos año de nacimiento
	Ext

In [116]:
data_frame_actores1.head(25)

,Nombre Intérprete,Año de nacimiento,Roles en la industria,Conocido por,Premios
0,Dennis Hopper,1936,Reparto. Dirección. Guion,Easy Rider (Buscando mi destino). Terciopelo a...,Nominado para 2 premios Óscar
1,Marley Shelton,1974,Reparto,Grindhouse: Planet Terror. Death Proof. Nunca ...,Otros premios
2,Devon Gummersall,octubre,Reparto. Dirección. Guion,Nashville. El secreto de la casa Blake. Es mi ...,Otros premios
3,Talia Shire,1946,Reparto. Producción. Dirección,Rocky. El padrino. Rocky IV. El padrino parte II,Nominado para 2 premios Óscar
4,Michael Cudlitz,1964,Reparto. Arte. Dirección,Un asesino algo especial. The Walking Dead. Se...,Otros premios
5,Peter Onorati,1953,Reparto. Guion. Producción,This Is Us. Uno de los nuestros. Cop Rock. El ...,Otros premios
6,Richard Portnow,0,Reparto. Producción. Equipos adicionales,Trumbo: La lista negra de Hollywood. Hitchcock...,Otros premios
7,Cheri Oteri,1962,Reparto. Guion. Producción,Scary Movie. Southland Tales. Mentiroso compul...,Nominado para 1 premio Primetime Emmy
8,Ryan Bollman,1972,Reparto,Los chicos del maíz 2: El sacrificio final. Só...,Otros premios
9,Joel Traywick,0,Reparto,Expediente X: Enfréntate al futuro. Skippy. Th...,No tiene premios


In [117]:
# lista_ids_pelis_2000 = ['tt0102362', 'tt0113092', 'tt0113086', 'tt0116748', 'tt0116391', 'tt0115937', 'tt0118694', 'tt0119495', 'tt0119806', 'tt0120202', 'tt0123003', 'tt0120263', 'tt0123055', 'tt0120630', 'tt0116628', 'tt0119273', 'tt0120755', 'tt0120913', 'tt0120673', 'tt0120903', 'tt0120917', 'tt0130623', 'tt0132910', 'tt0131704', 'tt0138749', 'tt0144964', 'tt0144574', 'tt0146984', 'tt0128158', 'tt0134084', 'tt0134983', 'tt0134847', 'tt0137115', 'tt0144084', 'tt0148120', 'tt0156020', 'tt0120753', 'tt0127349', 'tt0160495', 'tt0160644', 'tt0160289', 'tt0161081', 'tt0162711', 'tt0113026', 'tt0119866', 'tt0122459', 'tt0119966', 'tt0120467', 'tt0122161', 'tt0120117', 'tt0120698']

lista_identificadores = ['tt0122161', 'tt0120117', 'tt0120698']

data_frame_actores = extraer_datos_actores (lista_identificadores)


Maximizando la web.
Accediendo a la web.
Cookies aceptadas.
Entrando en la web de la película Falsa inocencia: https://www.imdb.com/title/tt0122161/
Explorando el casting
Extraigo el nombre y url de los actores
Entrando en la url de Dennis Hopper
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Marley Shelton
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Devon Gummersall
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Talia Shire
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Michael Cudlitz
	Extraemos año de nacimiento
	Ext

In [118]:
data_frame_actores.head(25)

,Nombre Intérprete,Año de nacimiento,Roles en la industria,Conocido por,Premios
0,Dennis Hopper,1936,Reparto. Dirección. Guion,Easy Rider (Buscando mi destino). Terciopelo a...,Nominado para 2 premios Óscar
1,Marley Shelton,1974,Reparto,Grindhouse: Planet Terror. Death Proof. Nunca ...,Otros premios
2,Devon Gummersall,octubre,Reparto. Dirección. Guion,Nashville. El secreto de la casa Blake. Es mi ...,Otros premios
3,Talia Shire,1946,Reparto. Producción. Dirección,Rocky. El padrino. Rocky IV. El padrino parte II,Nominado para 2 premios Óscar
4,Michael Cudlitz,1964,Reparto. Arte. Dirección,Un asesino algo especial. The Walking Dead. Se...,Otros premios
5,Peter Onorati,1953,Reparto. Guion. Producción,This Is Us. Uno de los nuestros. Cop Rock. El ...,Otros premios
6,Richard Portnow,0,Reparto. Producción. Equipos adicionales,Trumbo: La lista negra de Hollywood. Hitchcock...,Otros premios
7,Cheri Oteri,1962,Reparto. Guion. Producción,Scary Movie. Southland Tales. Mentiroso compul...,Nominado para 1 premio Primetime Emmy
8,Ryan Bollman,1972,Reparto,Los chicos del maíz 2: El sacrificio final. Só...,Otros premios
9,Joel Traywick,0,Reparto,Expediente X: Enfréntate al futuro. Skippy. Th...,No tiene premios


In [ ]:
data_frame_actores.to_csv("actores_peliculas_muestra.csv")

In [64]:
data_frame_actores.to_csv("actores_peliculas_2000.csv")